

# Building Content-Based Recommendation System using Cosine Similarity.¶




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/X/data - data - data - data.csv')
df.head(5)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# types of column data
df.dtypes

In [ ]:
df

# EDA

**Likes vs Dislikes**

In [ ]:
import altair as alt
chart = alt.Chart(df).mark_bar().encode(x='Likes', y='Dislikes', color='Category')
chart


Product Tags


In [ ]:
import matplotlib.pyplot as plt
df['Product Tags'].value_counts().head(10).plot(kind='barh')
plt.xlabel('Frequency')
_ = plt.ylabel('Product Tag')

Count of Records Vs Watch Time

In [ ]:


import altair as alt
alt.Chart(df).mark_bar().encode(
    x = alt.X('Watch Time (Seconds):Q', bin=alt.Bin(maxbins=20)),
    y = 'count()',color='Watch Time (Seconds)'
).properties(width=600)


In [ ]:
alt.Chart(df).mark_line().encode(
    x = 'Watch Time (Seconds)',
    y = 'count()'
).properties(width=600)



Categories based on Likes

In [ ]:
alt.Chart(df).mark_bar().encode(x='Category', y='Likes', color='Category').properties(height=400)


In [ ]:
# @title Likes, Dislikes, and Watch Time

import matplotlib.pyplot as plt
plt.scatter(df['Likes'], df['Dislikes'], c=df['Watch Time (Seconds)'], cmap='viridis')
plt.xlabel('Likes')
plt.ylabel('Dislikes')
plt.title('Likes, Dislikes, and Watch Time')
_ = plt.colorbar(label='Watch Time (Seconds)')

In [ ]:
# @title Dislikes vs Watch Time (Seconds)

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='Dislikes', y='Watch Time (Seconds)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Watch Time (Seconds)

from matplotlib import pyplot as plt
df['Watch Time (Seconds)'].plot(kind='line', figsize=(8, 4), title='Watch Time (Seconds)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Likes

from matplotlib import pyplot as plt
df['Likes'].plot(kind='hist', bins=20, title='Likes')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# prompt: Using dataframe df: Dislikes vs Watch Time (Seconds) different colors

import altair as alt

alt.Chart(df).mark_line().encode(
    alt.X('Dislikes'),
    alt.Y('Watch Time (Seconds)'),
    color='Category'
).interactive()


Product Tags

In [ ]:
from wordcloud import WordCloud

# Extract product tags from the dataframe
product_tags = " ".join(df["Product Tags"].tolist())

# Generate wordcloud
wordcloud = WordCloud(background_color="white",width=800, height=400).generate(product_tags)

# Display the wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


Product Tags and title

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine Product Tags and Title
df['Combined'] = df['Product Tags'] + ' ' + df['Title']

# Create a WordCloud object
wc = WordCloud(background_color="white", max_words=300, max_font_size=50)

# Generate the word cloud
wc.generate(df['Combined'].str.cat(sep=" "))

# Display the generated image
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")


In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine Product Tags and Title
df['Combined'] = df['Product Tags'] + ' ' + df['Title'] + df['Product Tags']

# Create a WordCloud object
wc = WordCloud(background_color="white", max_words=300, max_font_size=100)

# Generate the word cloud
wc.generate(df['Combined'].str.cat(sep=" "))

# Display the generated image
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")


Product tags, title, category

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine Product Tags and Title
df['Combined'] = df['Product Tags'] + ' ' + df['Title'] + df['Product Tags'] + ' '  + df['Category']

# Create a WordCloud object
wc = WordCloud(background_color="white", max_words=100, max_font_size=50)

# Generate the word cloud
wc.generate(df['Combined'].str.cat(sep=" "))

# Display the generated image
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

In [ ]:
df['Combined'] = df['Title'] + ' ' + df['Category'] + ' ' + df['Product Tags']
df['Combined'] = df['Combined'].str.lower()
df['Combined'] = df['Combined'].str.replace('[^\w\s]', '')
df['Combined'] = df['Combined'].str.strip()


In [ ]:


df['Product Tags'].value_counts().to_frame().reset_index().rename(columns={'index':'Product Tag', 'Product Tags':'Count'}).head(10)


In [ ]:
df.head(5)

In [ ]:
# prompt: Using dataframe df: normalize all the data type including text and floats and objects

import pandas as pd

def normalize_dataframe(df):
  """
  Normalizes all data types in a dataframe to string type.
  """
  for column in df.columns:
    df[column] = df[column].astype(str)
  return df

df = normalize_dataframe(df.copy())


In [ ]:
df.head()

In [ ]:
from typing import Dict, Text

In [ ]:
def separate(text):
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Remove text inside parentheses
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
    clean_text = cleaned.translate(str.maketrans('','', string.digits))
    return clean_text

In [ ]:
df

In [ ]:
df['Video ID'] = df['Video ID'].astype('object')

In [ ]:
# Convert the 'Video ID' column to object type
df['Video ID'] = df['Video ID'].astype('object')

# Remove the row with the problematic value
df = df[df['Video ID'] != 'easy bread baking for beginners food baking, bread']

# Convert the remaining columns to float type
df = df.astype('string')

In [ ]:
df

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained BERT model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

# Prepare the text data
text = df['Combined'].tolist()

# Encode the text
embeddings = model(text)

# Print the shape of the embeddings
print(embeddings.shape)


In [ ]:


import pickle

# Save the embeddings to a pickle file
with open('embeddings.pkl', 'wb') as f:
  pickle.dump(embeddings, f)


In [ ]:
null_df = df.isnull().sum()[df.isnull().sum() > 0].sort_values().to_frame().rename(columns={0: "Total Missing values"})
null_df["% of Missing Values"] = round(null_df["Total Missing values"]/len(df)*100, 2)
null_df["Feature Data Type"] = df[null_df.index.tolist()].dtypes

null_df

In [ ]:


# Extract features from the data
features = df.drop(['Video ID', 'Title', 'Category', 'Product Tags', 'Combined'], axis=1)

# Print the features
print(features.head())


In [ ]:
!pip install featuretools

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install --upgrade featuretools

In [ ]:
df

In [ ]:
df['Combined'] = df['Combined'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))


In [ ]:
vectorizer = TfidfVectorizer()
tfid_vectorizer = vectorizer.fit_transform(df['Combined'].apply(lambda x: ', '.join(x)))

In [ ]:
user_input = ['Artificial Intelligence', 'Desk', 'Workouts']  # Replace with any ingredeints
user_input_vector = vectorizer.transform([', '.join(user_input)])

cosine_similarities = cosine_similarity(tfid_vectorizer, user_input_vector).flatten()

top_5_indices = cosine_similarities.argsort()[-5:][::-1]

recommended_videos = df.iloc[top_5_indices]

recommended_titles = recommended_videos['Title']
print(recommended_titles)

In [ ]:


print("Recommended videos:")
for title in recommended_titles:
    print(f"* {title}")


In [ ]:


import pickle

with open('cosine_similarities.pkl', 'wb') as f:
  pickle.dump(cosine_similarities, f)
